# TSVD

In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD as skTSVD
from cuml import TruncatedSVD as cumlTSVD
import cudf
import os

# Helper Functions

In [2]:
from timeit import default_timer

class Timer(object):
    def __init__(self):
        self._timer = default_timer
    
    def __enter__(self):
        self.start()
        return self

    def __exit__(self, *args):
        self.stop()

    def start(self):
        """Start the timer."""
        self.start = self._timer()

    def stop(self):
        """Stop the timer. Calculate the interval in seconds."""
        self.end = self._timer()
        self.interval = self.end - self.start

In [3]:
# check if mortgage dataset is present and then extract the data from it, else throw an error
import gzip
# change the path of the mortgage dataset if you have saved it in a different directory
def load_data(nrows, ncols, cached = '/rapids/notebooks/wip/notebooks/cuml/data/mortgage.npy.gz'):
    train_rows = int(nrows*0.8)
    if os.path.exists(cached):
        print('use mortgage data')
        with gzip.open(cached) as f:
            X = np.load(f)
        # the 4th column is 'adj_remaining_months_to_maturity'
        # used as the label
        X = X[:,[i for i in range(X.shape[1]) if i!=4]]
        y = X[:,4:5]
        rindices = np.random.randint(0,X.shape[0]-1,nrows)
        X = X[rindices,:]
    else:
        # raise an exception if the dataset is not present
        raise FileNotFoundError('Please download the required dataset or check the path')
    df_X_train = pd.DataFrame({'fea%d'%i:X[0:train_rows,i] for i in range(X.shape[1])})
    df_X_test = pd.DataFrame({'fea%d'%i:X[train_rows:,i] for i in range(X.shape[1])})    
    return df_X_train, df_X_test

In [4]:
from sklearn.metrics import mean_squared_error
def array_equal(a,b,threshold=5e-3,with_sign=True):
    a = to_nparray(a)
    b = to_nparray(b)
    if with_sign == False:
        a,b = np.abs(a),np.abs(b)
    error = mean_squared_error(a,b)
    res = error<threshold
    return res

def to_nparray(x):
    if isinstance(x,np.ndarray) or isinstance(x,pd.DataFrame):
        return np.array(x)
    elif isinstance(x,np.float64):
        return np.array([x])
    elif isinstance(x,cudf.DataFrame) or isinstance(x,cudf.Series):
        return x.to_pandas().values
    return x

# Run tests

In [ ]:
%%time
nrows = 2**22
ncols = 40
# split the data into 80/20 split where 80% of the data is set as the training data and the remaining 20% is set as the testing data
X_train, X_test = load_data(nrows,ncols)
print('training data',X_train.shape)
print('testing data',X_test.shape)

use mortgage data


In [ ]:
n_components = 10
random_state = 42

In [ ]:
%%time
algorithm='arpack'
tsvd_sk = skTSVD(n_components=n_components,algorithm=algorithm, 
            random_state=random_state)
tsvd_sk.fit(X_train)
result_sk = tsvd_sk.transform(X_test)

In [ ]:
%%time
# convert the pandas dataframe to cudf format
X_cudf = cudf.DataFrame.from_pandas(X_train)
X_cudf_test = cudf.DataFrame.from_pandas(X_test)

In [ ]:
%%time
algorithm='full'
tsvd_cuml = cumlTSVD(n_components=n_components,algorithm=algorithm, 
            random_state=random_state)
tsvd_cuml.fit(X_cudf)
result_cuml = tsvd_cuml.transform(X_cudf_test)

In [ ]:
for attr in ['singular_values_','components_','explained_variance_','explained_variance_ratio_']:
    passed = array_equal(getattr(tsvd_sk,attr),getattr(tsvd_cuml,attr),threshold=0.1)
    # larger error margin due to different algorithms: arpack vs full
    message = 'compare tsvd: cuml vs sklearn {:>25} {}'.format(attr,'equal' if passed else 'NOT equal')
    print(message)

In [ ]:
passed = array_equal(result_sk,result_cuml,threshold=0.1)
# larger error margin due to different algorithms: arpack vs full
message = 'compare tsvd: cuml vs sklearn transformed results %s'%('equal'if passed else 'NOT equal')
print(message)

In [ ]:
# create the original data from the transformed data
recreated_data_sk = tsvd_sk.inverse_transform(result_sk)
recreated_data_cuml = tsvd_cuml.inverse_transform(result_cuml)

In [ ]:
passed_inv = array_equal(recreated_data_sk,recreated_data_cuml,threshold=0.1)
message = 'compare tsvd: cuml vs sklearn inverted results %s'%('equal'if passed_inv else 'NOT equal')
print(message)

In [ ]:
passed_check = array_equal(X_cudf_test,recreated_data_cuml,threshold=0.1)
message = 'compare tsvd: original dataset vs reduced dimensional results %s'%('equal'if passed_check else 'NOT equal')
print(message)

In [ ]:
from sklearn.utils.testing import (assert_array_almost_equal, assert_equal)
assert_array_almost_equal(tsvd_sk.singular_values_, to_nparray(tsvd_cuml.singular_values_), decimal=1)